In [ ]:
# # This is a temporary fix for geoplot
# !pip install cartopy==0.17.0
# !pip install geoplot

In [ ]:
# Import common libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D

# Import geo libraries
import urbanpy as up
import geopandas as gpd
import contextily as cx
import geoplot as gplt
from shapely.geometry import Polygon, Point

In [ ]:
gdf_ALL = gpd.read_file("outputs/datos_georef_completo.geojson", driver="GeoJSON")

In [ ]:
ica = up.download.nominatim_osm("Ica, Peru")
ica.crs = 4326

In [ ]:
ax = ica.plot(facecolor="none")
gdf_ALL.plot(ax=ax)

In [ ]:
gdf_ALL["Provincia donde vive"].value_counts()

## Create variables for making the maps

### Inseguridad Alimentaria

In [ ]:
rename_cols = {
    "En los últimos 12 meses ¿Usted u otra persona en su hogar se haya preocupado por no tener suficientes alimentos para comer por falta de dinero u otros recursos?": "Preocupación por no tener alimentos (Leve)",
    "Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar no haya podido comer alimentos saludables y nutritivos  por falta de dinero u otros recursos?": "No pudo comer alimentos saludables y nutritivos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya comido poca variedad de  alimentos por falta de dinero u otros recursos?": "Comió poca variedad de alimentos (Moderada)",
    "En los últimos 12 meses Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar haya comido menos de lo que pensaba que debía comer por falta de dinero u otros recursos?": "Comió menos de lo que pensaba que debía (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya tenido que dejar de desayunar, almorzar o cenar porque no había suficiente dinero u otros recursos para obtener alimentos?": "Se saltó alguna comida por falta de dinero para comprar alimentos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya sentido  hambre pero no comió porque no había suficiente dinero u otros recursos para obtener alimentos?": "Tuvo hambre pero no comió por falta de alientos (Moderada)",
    "En los últimos 12 meses ¿Hubo alguna vez en que su hogar se haya quedado sin alimentos por falta de dinero u otros recursos?": "Se quedó sin alimentos (Grave)",
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya dejado de comer todo un día por falta de dinero u otros recursos?": "Dejo de comer todo un día (Grave)",
}

In [ ]:
cols = list(rename_cols.values())
cols.reverse()

In [ ]:
gdf_ALL = gdf_ALL.rename(columns=rename_cols)

In [ ]:
def get_insec_alim(row):
    for c in cols:
        if row[c] == "Si":
            return c.split("(")[1][:-1]
    return np.nan

In [ ]:
gdf_ALL["inseg_alim_nivel"] = gdf_ALL.apply(get_insec_alim, axis=1)

In [ ]:
gdf_ALL["inseg_alim_nivel"].value_counts()

In [ ]:
gdf_ALL["inseg_alim_nivel"].isna().sum()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["inseg_alim_nivel"]).to_dict())

### Qaliwarma variables

In [ ]:
gdf_ALL[
    [
        "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?",
        "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?",
        "¿Que modalidad de Qali Warma recibe su hijo?",
    ]
].head()

In [ ]:
gdf_ALL[
    "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"
].value_counts()

In [ ]:
gdf_ALL[
    "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
].value_counts()

In [ ]:
print("Number of people that receive Qali Warma and consider it important: ")
gdf_ALL[
    (
        gdf_ALL[
            "Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"
        ]
        == "SÍ"
    )
    & (
        gdf_ALL[
            "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
        ]
        == "SÍ"
    )
].shape[0]

In [ ]:
gdf_ALL["tiene_qw"] = (
    gdf_ALL["Si tiene hijos en edad escolar, ¿Alguno de sus hijos recibe Qali Warma?"]
    == "SÍ"
).astype(int)
gdf_ALL["qw_importante"] = (
    gdf_ALL[
        "¿Los alimentos recibidos por Qali Warma rma son un aporte importante a la alimentacion familiar?"
    ]
    == "SÍ"
).astype(int)

### Hogares que estan dispuestos a organizarse en estrategias alimentacion colectiva

In [ ]:
gdf_ALL["En caso no, ¿Estaria dispuesto en participar en una?"].value_counts()

In [ ]:
gdf_ALL["dispuesto_organizarse"] = (
    gdf_ALL["En caso no, ¿Estaria dispuesto en participar en una?"] == "SÍ"
).astype(int)

### Estatus migratorio

In [ ]:
gdf_ALL["¿Cuál es SU actual ESTATUS MIGRATORIO?"].value_counts()

In [ ]:
gdf_ALL["status_migratorio"] = (
    gdf_ALL["¿Cuál es SU actual ESTATUS MIGRATORIO?"]
    .replace({"Otro": np.nan})
    .fillna(gdf_ALL["Especifica otro estatus migratorio"])
)

In [ ]:
gdf_ALL["status_migratorio"].value_counts()

In [ ]:
gdf_ALL["status_migratorio"].unique()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["status_migratorio"]).to_dict())

In [ ]:
# Check integrity of value counts
gdf_ALL[
    [
        "Carnet de Extranjeria",
        "CPP",
        "Mantengo mi documento de identidad venezolano",
        "No especificado",
        "Carnet de Solicitud de Refugio",
        "No posee documentos",
    ]
].sum()

### Promedio de ingreso familiar

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].value_counts()

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].isna().sum()

In [ ]:
gdf_ALL[
    "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
].unique()

In [ ]:
gdf_ALL["rango_ingreso"] = (
    gdf_ALL[
        "Sumando a TODOS los miembros de su hogar que perciben algun tipo de ingreso ¿Cuál es el INGRESO PROMEDIO MENSUAL  que reciben?"
    ]
    .replace({0: "0 a 300", None: np.nan})
    .fillna("Ingreso_No especificado")
)

In [ ]:
gdf_ALL["rango_ingreso"].unique()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["rango_ingreso"]).to_dict())

### Tipo de vivienda

In [ ]:
gdf_ALL["Tipo de vivienda"].value_counts()

In [ ]:
gdf_ALL["Tipo de vivienda"].isna().sum()

In [ ]:
gdf_ALL["Tipo de vivienda"].unique()

In [ ]:
gdf_ALL = gdf_ALL.assign(**pd.get_dummies(gdf_ALL["Tipo de vivienda"]).to_dict())

### Trabajo formal

In [ ]:
gdf_ALL["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"].value_counts()

In [ ]:
gdf_ALL["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"].unique()

In [ ]:
gdf_ALL["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"].isna().sum()

In [ ]:
gdf_ALL = gdf_ALL.assign(
    **pd.get_dummies(
        gdf_ALL["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"]
    )
    .rename({"No especifica": "TrabajoFormal_No especifica"}, axis=1)
    .to_dict()
)

### Densidad Poblacional

In [ ]:
peru_pop = up.download.hdx_dataset(
    "4e74db39-87f1-4383-9255-eaf8ebceb0c9/resource/317f1c39-8417-4bde-a076-99bd37feefce/download/per_general_2020_csv.zip"
)

In [ ]:
ica_pop = up.geom.filter_population(peru_pop, ica)

In [ ]:
ica_pop.head()

## Aggregation of data by H3 hexagon resolution 9 (0.10km^2)

In [ ]:
hexs_ica9 = up.geom.gen_hexagons(9, ica)

In [ ]:
hexs_ica9.shape

In [ ]:
hexs_ica9 = up.geom.merge_shape_hex(
    hexs_ica9,
    gdf_ALL,
    agg={
        "Grave": "sum",
        "Moderada": "sum",
        "Leve": "sum",
        "tiene_qw": "sum",
        "qw_importante": "sum",
        "dispuesto_organizarse": "sum",
        "Carnet de Extranjeria": "sum",
        "CPP": "sum",
        "Mantengo mi documento de identidad venezolano": "sum",
        "No especificado": "sum",
        "Carnet de Solicitud de Refugio": "sum",
        "No posee documentos": "sum",
        "0 a 300": "sum",
        "301 a 500": "sum",
        "501 a 1025 SALARIO BASICO": "sum",
        "1026 a 1500": "sum",
        "1501 A 2000": "sum",
        "2000 a más": "sum",
        "Ingreso_No especificado": "sum",
        "Cuarto en una casa o departamento (MAS DE 1 FAMILIA)": "sum",
        "Casa independiente (UNA SOLA FAMILIA)": "sum",
        "Departamento (UNA SOLA FAMILIA)": "sum",
        "Otro, especificar (Ejemplo: CHOZA DE ESTERAS)": "sum",
        "Vivienda en una quinta o callejón": "sum",
        "Refugio, casa hogar, otro similar": "sum",
        "NO": "sum",
        "No tengo trabajo": "sum",
        "TrabajoFormal_No especifica": "sum",
        "SÍ": "sum",
        "Sin respuesta": "sum",
    },
)

In [ ]:
hexs_ica9 = up.geom.merge_shape_hex(hexs_ica9, ica_pop, agg={"per_general_2020": "sum"})

In [ ]:
hexs_ica9.isna().sum() / hexs_ica9.shape[0]

## Definir zonas de analisis/estudio

In [ ]:
districts = gpd.read_file(
    "https://storage.googleapis.com/up_public_geodata/admin_bounds/peru/districts.zip",
    mask=ica,
)

In [ ]:
ica_districts = districts[districts["DEPARTAMEN"] == "ICA"]

In [ ]:
ax = ica.plot(facecolor="none")
ica_districts.plot(ax=ax)

In [ ]:
# ICA
gdf_ica = gdf_ALL.clip(
    ica_districts[ica_districts["PROVINCIA"] == "ICA"].unary_union
).cx[-75.77531:-75.67972, -14.15:-13.970591]

# CHINCHA
gdf_chincha = gdf_ALL.clip(
    ica_districts[
        (ica_districts["PROVINCIA"] == "CHINCHA")
        & (ica_districts["DISTRITO"] != "SAN PEDRO DE HUACARPANA")
    ].unary_union
).cx[-76.1818:-76.10, -13.45:-13.39094]

# PISCO
gdf_pisco = gdf_ALL.clip(
    ica_districts[ica_districts["PROVINCIA"] == "PISCO"].unary_union
).cx[-76.25:-76.1, -13.78:-13.65]

In [ ]:
print("Tamaño de la muestra original:", gdf_ALL.shape[0])
print("Tamaño de la muestra por zona de analisis")
print("Ica:", gdf_ica.shape[0])
print("Pisco:", gdf_pisco.shape[0])
print("Chincha:", gdf_chincha.shape[0])

In [ ]:
# Calculate value for the map scale
points = gpd.GeoSeries(
    [Point(-75, -13.75), Point(-76, -13.75)], crs=4326
)  # Geographic WGS 84 - degrees
points = points.to_crs(32618)  # Projected WGS 84 - meters
distance_meters = points[0].distance(points[1])

In [ ]:
def bbox_to_gdf(minx, miny, maxx, maxy):
    """Converts a bounding box coords to a GeoDataFrame"""
    geometry = [Polygon([[minx, miny], [minx, maxy], [maxx, maxy], [maxx, miny]])]
    return gpd.GeoSeries(geometry, crs=4326)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))

ica_districts[ica_districts["PROVINCIA"].isin(["CHINCHA", "PISCO", "ICA"])].plot(
    alpha=0.5, facecolor="none", label="Encuestas", legend=True, ax=ax
)

gdf_ica.plot(color="g", ax=ax)
gdf_chincha.plot(color="g", ax=ax)
gdf_pisco.plot(color="g", ax=ax)

bbox_to_gdf(*gdf_ica.total_bounds).plot(
    facecolor="none", edgecolor="r", linewidth=3, ax=ax
)
bbox_to_gdf(*gdf_chincha.total_bounds).plot(
    facecolor="none", edgecolor="orange", linewidth=3, ax=ax
)
bbox_to_gdf(*gdf_pisco.total_bounds).plot(
    facecolor="none", edgecolor="k", linewidth=3, ax=ax
)

minx, miny, maxx, maxy = -76.4, -14.25, -75.6, -13.25
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Observaciones")

r_line = Line2D([0], [0], color="r", linewidth=3, label="Ica")
b_line = Line2D([0], [0], color="k", linewidth=3, label="Pisco")
o_line = Line2D([0], [0], color="orange", linewidth=3, label="Chincha")

plt.legend(handles=[o_line, b_line, r_line, g_patch], title="Zonas de análisis")

ax.set_axis_off()
# cx.add_basemap(ax,  source=cx.providers.CartoDB.Positron, crs=4326, attribution=False)
cx.add_basemap(
    ax, source=cx.providers.Stamen.PositronNoLabels, crs=4326, attribution=False
)
cx.add_basemap(
    ax, source=cx.providers.Stamen.PositronOnlyLabels, crs=4326, attribution=False
)

plt.savefig("outputs/maps_v2/zonas_de_analisis.png", dpi=300, bbox_inches="tight")

## Mapas de inseguridad alimentaria para cada nivel 


In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Nivel de Inseguridad Alimentaria ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Nivel: {inseg_niveles[j]}")

        if j == 2:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                inseg_niveles[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=20,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                inseg_niveles[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=20,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        # cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs='EPSG:4326', attribution=False)
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_inseg_alim.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Nivel de Inseguridad Alimentaria ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Nivel: {inseg_niveles[j]}")

        gplt.kdeplot(
            gdf.query(f"inseg_alim_nivel == '{inseg_niveles[j]}'"),
            cmap="turbo",
            ax=ax,
            alpha=0.5,
            n_levels=100,
            fill=True,
            thresh=0.5,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        if j == 0:
            ax.add_artist(
                ScaleBar(distance_meters, box_alpha=0.5, location="lower left")
            )

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
            crs="EPSG:4326",
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_inseg_alim.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Point Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle(f"Nivel de Inseguridad Alimentaria")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
inseg_niveles = ["Grave", "Moderada", "Leve"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        gdfs[j].plot(
            "inseg_alim_nivel",
            alpha=1,
            cmap="Reds_r",
            ax=ax,
            categories=["Grave", "Moderada", "Leve"],
            legend=True,
            legend_kwds={"loc": "lower left", "bbox_to_anchor": (1, 0.5)},
        )
    else:
        gdfs[j].plot(
            "inseg_alim_nivel",
            alpha=1,
            cmap="Reds_r",
            ax=ax,
            categories=["Grave", "Moderada", "Leve"],
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax,
        source="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        crs="EPSG:4326",
        attribution=False,
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

plt.tight_layout()
plt.savefig(f"outputs/maps_v2/inseg_alim_puntos.png", dpi=300, bbox_inches="tight")

## Mapa de hogares que recibieron QW categorizado por si fue significativo o no

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
vars = ["tiene_qw", "qw_importante"]

for i, gdf in enumerate(gdfs):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    fig.suptitle(f"{ciudades[i]}")

    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)

    minx, miny, maxx, maxy = gdf.total_bounds

    ax.set_title(
        f"Familias que reciben Qali Warma y es\nimportante para la alimentación familiar"
    )

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    hexs_ica9.cx[minx:maxx, miny:maxy].plot(
        "qw_importante",
        edgecolor="none",
        alpha=0.75,
        cmap="Reds",
        ax=ax,
        vmin=0,
        vmax=10,
        legend=True,
        cax=cax,
    )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326, attribution=False
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_qaliwarma.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
vars = ["tiene_qw", "qw_importante"]
labels = [
    "Familias que reciben Qaliwarma",
    "Los alimentos de Qaliwarma son importantes\npara la alimentación familiar",
]

for i, gdf in enumerate(gdfs):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    fig.suptitle(f"{ciudades[i]}")

    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)

    minx, miny, maxx, maxy = gdf.total_bounds

    ax.set_title(
        "Familias que reciben Qali Warma y es\nimportante para la alimentación familiar"
    )

    gplt.kdeplot(
        gdf.query(f"qw_importante == 1"),
        cmap="Reds",
        ax=ax,
        alpha=0.5,
        fill=True,
        n_levels=100,
        thresh=0.5,
    )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326, attribution=False
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_qaliwarma.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa con personas que estan dispuestas a organizarse 

In [ ]:
# Hex Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa con personas que estan dispuestas a organizarse")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        hexs_ica9.cx[minx:maxx, miny:maxy].plot(
            "dispuesto_organizarse",
            alpha=0.75,
            cmap="Reds",
            ax=ax,
            vmin=0,
            vmax=30,
            legend=True,
            cax=cax,
            edgecolor="none",
        )
    else:
        hexs_ica9.cx[minx:maxx, miny:maxy].plot(
            "dispuesto_organizarse",
            alpha=0.75,
            cmap="Reds",
            ax=ax,
            vmin=0,
            vmax=30,
            edgecolor="none",
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326, attribution=False
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

plt.tight_layout()
plt.savefig(f"outputs/maps_v2/dispuestas_organizarse.png", dpi=300, bbox_inches="tight")

In [ ]:
# Heat Map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa con personas que estan dispuestas a organizarse")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    gplt.kdeplot(
        gdfs[j].query(f"dispuesto_organizarse == 1"),
        cmap="Reds",
        ax=ax,
        alpha=0.5,
        fill=True,
        n_levels=100,
        thresh=0.5,
    )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    # Add scale bar
    ax.add_artist(ScaleBar(distance_meters, box_alpha=0.5, location="lower left"))

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326, attribution=False
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

plt.tight_layout()
plt.savefig(
    f"outputs/maps_v2/density/dispuestas_organizarse.png", dpi=300, bbox_inches="tight"
)

## Mapa con hogares por tipo de situación migratoria

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
status_migratorio_options = [
    "Carnet de Extranjeria",
    "CPP",
    "Carnet de Solicitud de Refugio",
    "Mantengo mi documento de identidad venezolano",
    "No posee documentos",
    "No especificado",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Estatus migratorio ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Estatus: {status_migratorio_options[j]}")

        if j == 2:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                status_migratorio_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                status_migratorio_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_status_migratorio.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
status_migratorio_options = [
    "Carnet de Extranjeria",
    "CPP",
    "Carnet de Solicitud de Refugio",
    "Mantengo mi documento de identidad venezolano",
    "No posee documentos",
    "No especificado",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Estatus migratorio ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Estatus: {status_migratorio_options[j]}")

        try:
            gplt.kdeplot(
                gdf.query(f"status_migratorio == '{status_migratorio_options[j]}'"),
                cmap="Reds",
                ax=ax,
                alpha=0.5,
                fill=True,
                n_levels=100,
                thresh=0.5,
            )
        except Exception as e:
            print(e)
            continue

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_status_migratorio.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa con hogares por ingreso 

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
ingreso_options = [
    "0 a 300",
    "301 a 500",
    "501 a 1025 SALARIO BASICO",
    "1026 a 1500",
    "1501 A 2000",
    "2000 a más",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Ingreso promedio en Soles (S/) - ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Ingreso promedio: {ingreso_options[j]}")

        if j == 5:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                ingreso_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                ingreso_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_ingreso_promedio.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
ingreso_options = [
    "0 a 300",
    "301 a 500",
    "501 a 1025 SALARIO BASICO",
    "1026 a 1500",
    "1501 A 2000",
    "2000 a más",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Ingreso promedio en Soles (S/) - ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Ingreso promedio: {ingreso_options[j]}")

        gplt.kdeplot(
            gdf.query(f"rango_ingreso == '{ingreso_options[j]}'"),
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            n_levels=100,
            thresh=0.5,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_ingreso_promedio.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa de densidad poblacional

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Mapa de densidad poblacional (hab./0.1km$^2$)")

gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]

for j, ax in enumerate(axes):
    ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
    minx, miny, maxx, maxy = gdfs[j].total_bounds

    ax.set_title(f"Ciudad: {ciudades[j]}")

    if j == 2:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        hexs_ica9.cx[minx:maxx, miny:maxy].query("per_general_2020 > 0").plot(
            "per_general_2020",
            alpha=0.6,
            ax=ax,
            cmap="Reds",
            vmin=0,
            vmax=2000,
            legend=True,
            cax=cax,
            edgecolor="none",
        )
    else:
        hexs_ica9.cx[minx:maxx, miny:maxy].query("per_general_2020 > 0").plot(
            "per_general_2020",
            alpha=0.6,
            ax=ax,
            cmap="Reds",
            vmin=0,
            vmax=2000,
            edgecolor="none",
        )

    margins = 0.001
    ax.set_xlim(minx - margins, maxx + margins)
    ax.set_ylim(miny - margins, maxy + margins)

    ax.set_axis_off()
    cx.add_basemap(
        ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326, attribution=False
    )
    cx.add_basemap(
        ax=ax,
        source=cx.providers.CartoDB.PositronOnlyLabels,
        crs=4326,
        attribution=False,
    )

plt.tight_layout()
plt.savefig(f"outputs/maps_v2/densidad_poblacional.png", dpi=300, bbox_inches="tight")

## Mapa de tipo de vivienda

In [ ]:
gdf_ALL["Tipo de vivienda"].unique()

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
tipo_viv_options = [
    "Casa independiente (UNA SOLA FAMILIA)",
    "Departamento (UNA SOLA FAMILIA)",
    "Cuarto en una casa o departamento (MAS DE 1 FAMILIA)",
    "Vivienda en una quinta o callejón",
    "Otro, especificar (Ejemplo: CHOZA DE ESTERAS)",
    "Refugio, casa hogar, otro similar",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Tipo de vivienda ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"{tipo_viv_options[j]}")

        if j == 5:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                tipo_viv_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                tipo_viv_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_tipo_vivienda.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
tipo_viv_options = [
    "Casa independiente (UNA SOLA FAMILIA)",
    "Departamento (UNA SOLA FAMILIA)",
    "Cuarto en una casa o departamento (MAS DE 1 FAMILIA)",
    "Vivienda en una quinta o callejón",
    "Otro, especificar (Ejemplo: CHOZA DE ESTERAS)",
    "Refugio, casa hogar, otro similar",
]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Tipo de vivienda ({ciudades[i]})")
    for j, ax in enumerate(axes.flatten()):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"{tipo_viv_options[j]}")

        gplt.kdeplot(
            gdf[gdf["Tipo de vivienda"] == tipo_viv_options[j]],
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            n_levels=100,
            thresh=0.5,
        )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_tipo_vivienda.png",
        dpi=300,
        bbox_inches="tight",
    )

## Mapa de Trabajo Formal

In [ ]:
gdf_ALL["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"].unique()

In [ ]:
# Hex Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
trab_formal_options = ["SÍ", "NO", "No tengo trabajo"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Trabajo Formal ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"{trab_formal_options[j]}")

        if j == 2:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                trab_formal_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
                legend=True,
                cax=cax,
            )
        else:
            hexs_ica9.cx[minx:maxx, miny:maxy].plot(
                trab_formal_options[j],
                edgecolor="none",
                alpha=0.75,
                cmap="Reds",
                ax=ax,
                vmin=0,
                vmax=15,
            )

        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/{ciudades[i]}_trabajo_formal.png",
        dpi=300,
        bbox_inches="tight",
    )

In [ ]:
# Heat Map
gdfs = [gdf_ica, gdf_chincha, gdf_pisco]
ciudades = ["Ica", "Chincha", "Pisco"]
trab_formal_options = ["SÍ", "NO", "No tengo trabajo"]

for i, gdf in enumerate(gdfs):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Trabajo Formal ({ciudades[i]})")
    for j, ax in enumerate(axes):
        ica_districts.plot(facecolor="none", edgecolor="lightgrey", ax=ax)
        minx, miny, maxx, maxy = gdf.total_bounds

        ax.set_title(f"Trabajo Formal: {trab_formal_options[j]}")

        gplt.kdeplot(
            gdf[gdf["Tipo de vivienda"] == tipo_viv_options[j]],
            cmap="Reds",
            ax=ax,
            alpha=0.5,
            fill=True,
            n_levels=100,
            thresh=0.5,
        )
        margins = 0.001
        ax.set_xlim(minx - margins, maxx + margins)
        ax.set_ylim(miny - margins, maxy + margins)

        ax.set_axis_off()
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronNoLabels,
            crs=4326,
            attribution=False,
        )
        cx.add_basemap(
            ax=ax,
            source=cx.providers.CartoDB.PositronOnlyLabels,
            crs=4326,
            attribution=False,
        )

    plt.tight_layout()
    plt.savefig(
        f"outputs/maps_v2/density/{ciudades[i]}_trabajo_formal.png",
        dpi=300,
        bbox_inches="tight",
    )